In [1]:
import sigpde.torch as sig
import torch
import math
device = torch.device('cuda:0')

In [2]:
kernel = sig.SigPDE(sig.kernels.LinearKernel(), 3)
kernel2 = sig.RobustSigPDE(sig.kernels.LinearKernel(), 3)

In [3]:
def generate(batch_size, length, dimension, device = torch.device('cpu')):
  random_walks = torch.randn(batch_size, length, dimension, dtype = torch.double, device = device) / math.sqrt(length)
  start = torch.zeros([batch_size, 1, dimension], device=device, dtype=torch.double)
  random_walks = torch.cat((start, random_walks), dim=1)
  random_walks = torch.cumsum(random_walks, dim=1)
  return random_walks

def add_time(x, start=0, stop=1):
    device = x.device
    dtype = x.dtype

    l = x.shape[1]

    t = torch.linspace(start, stop, l, device=device, dtype=dtype)
    t = t.unsqueeze(0).unsqueeze(-1)
    return torch.cat((x, t.expand(x.shape[0], x.shape[1], 1)), dim=-1)

def std_norm(x):
    return 2 - 1 / (1 + x.log())

def time_norm(x, time=True):
    if time:
        x = add_time(x)
    return x / x.pow(2).sum(dim=2).sqrt().max(dim=1).values.view(x.shape[0], 1, 1)

In [113]:
x = generate(1, 512, 50, device=device) * 3
y = generate(1, 512, 50, device=device)

In [114]:
x = time_norm(x)
y = time_norm(y)

In [115]:
kernel.pairwise(x)

tensor([2.8926], device='cuda:0', dtype=torch.float64)

In [116]:
c = std_norm(kernel.pairwise(x))
ff = lambda s : kernel.pairwise(x, x_scale=s) - c

In [60]:
ff(torch.tensor([1.0], device=x.device, dtype=x.dtype))

tensor([1.1353], device='cuda:0', dtype=torch.float64)

In [61]:
def bisection_single_eval(f, a, b, tol=1e-6, max_iter=100):
    fa = f(a)
    fb = f(b)
    k = 0
    if fa * fb >= 0:
        raise ValueError("Function values at the endpoints must have opposite signs.")
    
    for _ in range(max_iter):
        k += 1
        # Compute midpoint
        c = (a + b) / 2
        fc = f(c)

        # Stopping criterion based on function value
        if abs(fc) < tol:
            return c, fc, k

        # Update the interval and reuse function evaluations
        if fa * fc < 0:
            b, fb = c, fc  # Update the right endpoint
        else:
            a, fa = c, fc  # Update the left endpoint

    raise RuntimeError("Maximum number of iterations reached without convergence.")

In [84]:
def itp_single_eval(f, a=0, b=1,tol=1e-6, max_iter=100):
    fa = f(a)
    fb = f(b)
    k = 0

    if fa * fb >= 0:
        raise ValueError("Function values at the endpoints must have opposite signs.")
    
    for _ in range(max_iter):
        k += 1
        # Midpoint and interpolation calculation
        mid = (a + b) / 2
        interp = (a * fb - b * fa) / (fb - fa)
        t = 0.2  # Blending parameter (adjust if necessary)
        x = (1 - t) * mid + t * interp

        fx = f(x)

        # Update the interval and reuse function evaluations
        if fx * fa < 0:
            b, fb = x, fx  # Update the right endpoint
        else:
            a, fa = x, fx  # Update the left endpoint
            
        if abs(fx) < tol:  # Stopping based on function value
            print(k)
            return a, b
            
    return (a + b) / 2, k

In [71]:
def secant_single_eval(f, x0, x1, tol=1e-6, max_iter=100):
    f0 = f(x0)
    f1 = f(x1)
    k = 0

    if abs(f0) < tol:
        return x0
    if abs(f1) < tol:
        return x1

    for _ in range(max_iter):
        k += 1
        # Compute the next point using the secant formula
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        f2 = f(x2)

        # Check stopping criteria
        if abs(f2) < tol:
            return x2, f2, k

        # Update for the next iteration
        x0, f0 = x1, f1
        x1, f1 = x2, f2

In [117]:
a = torch.tensor([0], device=x.device, dtype=x.dtype)
b = torch.tensor([1], device=x.device, dtype=x.dtype)

In [118]:
itp_single_eval(ff, a=a, b=b)

15


(tensor([0.4117], device='cuda:0', dtype=torch.float64),
 tensor([0.4117], device='cuda:0', dtype=torch.float64))

In [106]:
bisection_single_eval(ff, a=a, b=b)

(tensor([0.0019], device='cuda:0', dtype=torch.float64),
 tensor([7.7386e-08], device='cuda:0', dtype=torch.float64),
 28)

In [119]:
a = torch.tensor([0], device=x.device, dtype=x.dtype)
b = torch.tensor([1], device=x.device, dtype=x.dtype)

In [120]:
a, b = itp_single_eval(ff, a=a, b=b, tol=0.01)
secant_single_eval(ff, a, b, tol=1e-6, max_iter=100)

4


(tensor([0.4117], device='cuda:0', dtype=torch.float64),
 tensor([-6.3920e-07], device='cuda:0', dtype=torch.float64),
 2)

tensor([0.0019], device='cuda:0', dtype=torch.float64)